In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly
from datetime import datetime
import datetime
import statsmodels.api as sm 
from sklearn.linear_model import LinearRegression
import math
import numpy as np
import urllib3
import sys
import matplotlib.pyplot as plt
import jinja2

In [ ]:
df = pd.read_excel('/Users/lukebarrett/Desktop/STA/STA 141B/New_ZEV_Sales_Last_updated_10-18-2022.xlsx')

In [ ]:
df.head(n=6)

,Data Year,County,FUEL_TYPE,MAKE,MODEL,Number of Vehicles
0,1998,Los Angeles,Electric,Ford,Ranger,1
1,1998,Orange,Electric,Ford,Ranger,1
2,1998,San Bernardino,Electric,Ford,Ranger,2
3,1998,San Mateo,Electric,Ford,Ranger,1
4,1999,Santa Barbara,Electric,Ford,Ranger,1
5,2000,Out Of State,Electric,Ford,Ranger,2


In [ ]:
out_of_state = df[ (df['County'] == 'Out Of State') ].index
df.drop(out_of_state , inplace=True)
df.head(n=6)

,Data Year,County,FUEL_TYPE,MAKE,MODEL,Number of Vehicles
0,1998,Los Angeles,Electric,Ford,Ranger,1
1,1998,Orange,Electric,Ford,Ranger,1
2,1998,San Bernardino,Electric,Ford,Ranger,2
3,1998,San Mateo,Electric,Ford,Ranger,1
4,1999,Santa Barbara,Electric,Ford,Ranger,1
6,2001,Los Angeles,Electric,Ford,Ranger,1


In [ ]:
year_df = df.groupby(['Data Year','County']).sum(numeric_only = True)
year_df.reset_index(inplace=True)
year_df

,Data Year,County,Number of Vehicles
0,1998,Los Angeles,1
1,1998,Orange,1
2,1998,San Bernardino,2
3,1998,San Mateo,1
4,1999,Santa Barbara,1
...,...,...,...
717,2022,Tulare,593
718,2022,Tuolumne,95
719,2022,Ventura,5583
720,2022,Yolo,907


In [ ]:
year_df.rename(columns = {'Data Year':'Year','Number of Vehicles':'Total Purchased EV'},inplace=True)
fig = px.bar(year_df,
             x=year_df['Year'], 
             y='Total Purchased EV', 
             #text_auto=True,
             log_x = True,
             color = 'County')
fig.update_xaxes(
        tickangle = 90,
        title_text = "Year",
        title_font = {"size": 20},
        title_standoff = 25)
fig.update_yaxes(
        tickangle = 90,
        title_text = "Total Purchased EV",
        title_font = {"size": 20},
        title_standoff = 25)
#fig.update_layout(title_text='Total Electric Vehicles Purchased by Year', title_x=0.5)
fig['layout']['title']['font'] = dict(size=25)
fig.update_layout()
fig.show()

In [ ]:
fig2 = px.pie(year_df, 
              values='Total Purchased EV',
              names='County')
#fig2.update_layout(title_text='Percentage of Electric Vehicles Purchased by County', title_x=0.12)
fig2.update_traces(textposition='inside')
fig2.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig2['layout']['title']['font'] = dict(size=25)
fig2.show()

In [ ]:
df2 = pd.read_csv('/Users/lukebarrett/Desktop/STA/STA 141B/STA 141B - Final Project/alt_fuel_stations_Oct_21_2022.csv')

In [ ]:
df2['Open Date'].tail()

In [ ]:
#convert object to datetime object
df2['Open Date'] = pd.to_datetime(df2['Open Date'], format="%Y-%m-%d")

In [ ]:
#extract only the year
charger_year = pd.DatetimeIndex(df2['Open Date']).year
charger_year

In [ ]:
charger_year = pd.DataFrame(charger_year)
charger_year.dropna()
charger_year.astype('Int64')

In [ ]:
charger_year_count = charger_year['Open Date'].value_counts()

In [ ]:
charger_year_count = pd.DataFrame(charger_year_count)
charger_year_count.index = charger_year_count.index.astype('Int64')
charger_year_count = charger_year_count.rename(columns={'Open Date':'Total Chargers'})
charger_year_count = charger_year_count.sort_index()
charger_year_count.head()

In [ ]:
year_df.drop('County', axis=1, inplace=True) 
year_df.head()

In [ ]:
year_df = year_df.groupby('Year')['Total Purchased EV'].sum()

In [ ]:
year_df = pd.DataFrame(year_df)
year_df = year_df.sort_index()

year_df.head()

In [ ]:
merged_df = pd.merge(year_df, charger_year_count, left_index=True, right_index=True)
# merged_df.index.names = ['Year']
# merged_df['index'] = merged_df.index
merged_df.head()

In [ ]:
dta = merged_df[['Total Purchased EV','Total Chargers']]
dta = sm.add_constant(dta) #constant to fit intercept of linear model

In [ ]:
ols = sm.OLS(dta['Total Purchased EV'], dta.drop(columns = 'Total Purchased EV')) #Initialize the OLS 
ols_res = ols.fit() # ols fitting

In [ ]:
print(ols_res.summary())

In [ ]:
ols_res.rsquared

In [ ]:
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
sns.lmplot(x ="Total Purchased EV", y ="Total Chargers", data = merged_df, order = 2, ci = None)

sns.regplot(data=merged_df, x='Total Purchased EV', y='Total Chargers', scatter = True, color = 'red')